In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part_number.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                url = 'https://ecatalog.smpcorp.com/v2/fs/api/part/partselect?part=' + input_.loc[a, 'webBase'] + '&func=PART&vid='
                
                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                vehicle = '\n'.join([guide['buyersGuideDesc'].strip() for guide in json_data['pp']['buyersGuides']])

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'rowId']),
                                         'Part_Number': json_data['pd']['webBase'].strip(),
                                         'Category': '' if len(json_data['pp']['categories'])==0 else json_data['pp']['categories'][0]['categoryName_en'].strip(),
                                         'Part_Type_Short': json_data['pd']['partDesc_en'].strip(),
                                         'Part_Type_Long': json_data['pd']['partLongDesc_en'].strip(),
                                         'Brand': json_data['pd']['brand'].strip(),
                                         'POP': json_data['pd']['pop'].strip(),
                                         'Per_Car': json_data['pd']['qtyEach'],
                                         'Vehicle': vehicle,
                                         'Pic': '',
                                         'Url': url,
                                         'Src': '',
                                         'JOIN_PNB': json_data['pd']['webBase'].strip() + ';' + json_data['pd']['brand'].strip()}])

                for specification in json_data['pp']['partSpecs']:
                    df_temp.loc[0, specification['attributeName_en'].lower().strip()] = specification['siteValue_en']

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'rowId': int(input_.loc[a, 'rowId']),
                                     'webBase': input_.loc[a, 'webBase']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'webBase'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['webBase'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：3242

[ok] - 15102
[尝试次数：1] - [剩余数量：3222] - [当前时间：14:36:29]

[ok] - 14232
[尝试次数：1] - [剩余数量：3221] - [当前时间：14:36:29]

[ok] - 14262
[尝试次数：1] - [剩余数量：3220] - [当前时间：14:36:29]

[ok] - 15106
[尝试次数：1] - [剩余数量：3219] - [当前时间：14:36:29]

[ok] - 11012
[尝试次数：1] - [剩余数量：3218] - [当前时间：14:36:29]

[ok] - 14235
[尝试次数：1] - [剩余数量：3217] - [当前时间：14:36:29]

[ok] - 15105
[尝试次数：1] - [剩余数量：3216] - [当前时间：14:36:29]

[ok] - 15104
[尝试次数：1] - [剩余数量：3215] - [当前时间：14:36:29]

[ok] - 15107
[尝试次数：1] - [剩余数量：3214] - [当前时间：14:36:29]

[ok] - 14233
[尝试次数：1] - [剩余数量：3213] - [当前时间：14:36:29]

[ok] - 11068
[尝试次数：1] - [剩余数量：3212] - [当前时间：14:36:29]

[ok] - 15103
[尝试次数：1] - [剩余数量：3211] - [当前时间：14:36:29]

[ok] - 15109
[尝试次数：1] - [剩余数量：3210] - [当前时间：14:36:29]

[ok] - 14234
[尝试次数：1] - [剩余数量：3209] - [当前时间：14:36:29]

[ok] - 14264
[尝试次数：1] - [剩余数量：3208] - [当前时间：14:36:29]

[ok] - 15100
[尝试次数：1] - [剩余数量：3207] - [当前时间：14:36:29]

[ok] - 15101
[尝试次数：1] - [剩余数量：3206] - [当前时间：14:36:29]

[ok] - 14261
[尝试次数：1] - [剩余数量：3205] - [当前时间：14:36:29]



[ok] - 55051
[尝试次数：1] - [剩余数量：3073] - [当前时间：14:36:35]

[ok] - 55055
[尝试次数：1] - [剩余数量：3072] - [当前时间：14:36:35]

[ok] - 55061
[尝试次数：1] - [剩余数量：3071] - [当前时间：14:36:35]

[ok] - 55058
[尝试次数：1] - [剩余数量：3070] - [当前时间：14:36:35]

[ok] - 55069
[尝试次数：1] - [剩余数量：3069] - [当前时间：14:36:35]

[ok] - 55066
[尝试次数：1] - [剩余数量：3068] - [当前时间：14:36:35]

[ok] - 55063
[尝试次数：1] - [剩余数量：3067] - [当前时间：14:36:35]

[ok] - 55064
[尝试次数：1] - [剩余数量：3066] - [当前时间：14:36:35]

[ok] - 55067
[尝试次数：1] - [剩余数量：3065] - [当前时间：14:36:35]

[ok] - 55071
[尝试次数：1] - [剩余数量：3064] - [当前时间：14:36:36]

[ok] - 55068
[尝试次数：1] - [剩余数量：3063] - [当前时间：14:36:36]

[ok] - 55073
[尝试次数：1] - [剩余数量：3062] - [当前时间：14:36:36]

[ok] - 55076
[尝试次数：1] - [剩余数量：3061] - [当前时间：14:36:36]

[ok] - 55077
[尝试次数：1] - [剩余数量：3060] - [当前时间：14:36:36]

[ok] - 55074
[尝试次数：1] - [剩余数量：3059] - [当前时间：14:36:36]

[ok] - 55078
[尝试次数：1] - [剩余数量：3058] - [当前时间：14:36:36]

[ok] - 55079
[尝试次数：1] - [剩余数量：3057] - [当前时间：14:36:36]

[ok] - 55081
[尝试次数：1] - [剩余数量：3056] - [当前时间：14:36:36]

[ok] - 550

[ok] - 55234
[尝试次数：1] - [剩余数量：2918] - [当前时间：14:36:43]

[ok] - 55243
[尝试次数：1] - [剩余数量：2917] - [当前时间：14:36:43]

[ok] - 55233
[尝试次数：1] - [剩余数量：2916] - [当前时间：14:36:43]

[ok] - 55239
[尝试次数：1] - [剩余数量：2915] - [当前时间：14:36:43]

[ok] - 55237
[尝试次数：1] - [剩余数量：2914] - [当前时间：14:36:43]

[ok] - 55256
[尝试次数：1] - [剩余数量：2913] - [当前时间：14:36:43]

[ok] - 55255
[尝试次数：1] - [剩余数量：2912] - [当前时间：14:36:43]

[ok] - 55253
[尝试次数：1] - [剩余数量：2911] - [当前时间：14:36:43]

[ok] - 55251
[尝试次数：1] - [剩余数量：2910] - [当前时间：14:36:43]

[ok] - 55268
[尝试次数：1] - [剩余数量：2909] - [当前时间：14:36:43]

[ok] - 55266
[尝试次数：1] - [剩余数量：2908] - [当前时间：14:36:43]

[ok] - 55272
[尝试次数：1] - [剩余数量：2907] - [当前时间：14:36:43]

[ok] - 55257
[尝试次数：1] - [剩余数量：2906] - [当前时间：14:36:44]

[ok] - 55271
[尝试次数：1] - [剩余数量：2905] - [当前时间：14:36:44]

[ok] - 55263
[尝试次数：1] - [剩余数量：2904] - [当前时间：14:36:44]

[ok] - 55246
[尝试次数：1] - [剩余数量：2903] - [当前时间：14:36:44]

[ok] - 55259
[尝试次数：1] - [剩余数量：2902] - [当前时间：14:36:44]

[ok] - 55261
[尝试次数：1] - [剩余数量：2901] - [当前时间：14:36:44]

[ok] - 552

[ok] - 55438
[尝试次数：1] - [剩余数量：2762] - [当前时间：14:36:51]

[ok] - 55436
[尝试次数：1] - [剩余数量：2761] - [当前时间：14:36:51]

[ok] - 55428
[尝试次数：1] - [剩余数量：2760] - [当前时间：14:36:51]

[ok] - 55446
[尝试次数：1] - [剩余数量：2759] - [当前时间：14:36:51]

[ok] - 55430
[尝试次数：1] - [剩余数量：2758] - [当前时间：14:36:51]

[ok] - 55444
[尝试次数：1] - [剩余数量：2757] - [当前时间：14:36:51]

[ok] - 55432
[尝试次数：1] - [剩余数量：2756] - [当前时间：14:36:51]

[ok] - 55437
[尝试次数：1] - [剩余数量：2755] - [当前时间：14:36:51]

[ok] - 55435
[尝试次数：1] - [剩余数量：2754] - [当前时间：14:36:51]

[ok] - 55439
[尝试次数：1] - [剩余数量：2753] - [当前时间：14:36:51]

[ok] - 55443
[尝试次数：1] - [剩余数量：2752] - [当前时间：14:36:51]

[ok] - 55454
[尝试次数：1] - [剩余数量：2751] - [当前时间：14:36:52]

[ok] - 55460
[尝试次数：1] - [剩余数量：2750] - [当前时间：14:36:52]

[ok] - 55455
[尝试次数：1] - [剩余数量：2749] - [当前时间：14:36:52]

[ok] - 55471
[尝试次数：1] - [剩余数量：2748] - [当前时间：14:36:52]

[ok] - 55453
[尝试次数：1] - [剩余数量：2747] - [当前时间：14:36:52]

[ok] - 55450
[尝试次数：1] - [剩余数量：2746] - [当前时间：14:36:52]

[ok] - 55451
[尝试次数：1] - [剩余数量：2745] - [当前时间：14:36:52]

[ok] - 554

[ok] - 55691
[尝试次数：1] - [剩余数量：2607] - [当前时间：14:37:00]

[ok] - 55676
[尝试次数：1] - [剩余数量：2606] - [当前时间：14:37:00]

[ok] - 55682
[尝试次数：1] - [剩余数量：2605] - [当前时间：14:37:00]

[ok] - 55687
[尝试次数：1] - [剩余数量：2604] - [当前时间：14:37:00]

[ok] - 55686
[尝试次数：1] - [剩余数量：2603] - [当前时间：14:37:00]

[ok] - 55689
[尝试次数：1] - [剩余数量：2602] - [当前时间：14:37:00]

[ok] - 55684
[尝试次数：1] - [剩余数量：2601] - [当前时间：14:37:00]

[ok] - 55692
[尝试次数：1] - [剩余数量：2600] - [当前时间：14:37:00]

[ok] - 55693
[尝试次数：1] - [剩余数量：2599] - [当前时间：14:37:00]

[ok] - 55679
[尝试次数：1] - [剩余数量：2598] - [当前时间：14:37:00]

[ok] - 55694
[尝试次数：1] - [剩余数量：2597] - [当前时间：14:37:00]

[ok] - 55685
[尝试次数：1] - [剩余数量：2596] - [当前时间：14:37:00]

[ok] - 55704
[尝试次数：1] - [剩余数量：2595] - [当前时间：14:37:00]

[ok] - 55695
[尝试次数：1] - [剩余数量：2594] - [当前时间：14:37:00]

[ok] - 55707
[尝试次数：1] - [剩余数量：2593] - [当前时间：14:37:00]

[ok] - 55703
[尝试次数：1] - [剩余数量：2592] - [当前时间：14:37:00]

[ok] - 55696
[尝试次数：1] - [剩余数量：2591] - [当前时间：14:37:00]

[ok] - 55700
[尝试次数：1] - [剩余数量：2590] - [当前时间：14:37:01]

[ok] - 557

...

[ok] - 66382
[尝试次数：1] - [剩余数量：631] - [当前时间：14:39:10]

[ok] - 66385
[尝试次数：1] - [剩余数量：630] - [当前时间：14:39:10]

[ok] - 66395
[尝试次数：1] - [剩余数量：629] - [当前时间：14:39:10]

[ok] - 66383
[尝试次数：1] - [剩余数量：628] - [当前时间：14:39:10]

[ok] - 66390
[尝试次数：1] - [剩余数量：627] - [当前时间：14:39:10]

[ok] - 66397
[尝试次数：1] - [剩余数量：626] - [当前时间：14:39:10]

[ok] - 66389
[尝试次数：1] - [剩余数量：625] - [当前时间：14:39:10]

[ok] - 66391
[尝试次数：1] - [剩余数量：624] - [当前时间：14:39:10]

[ok] - 66394
[尝试次数：1] - [剩余数量：623] - [当前时间：14:39:10]

[ok] - 66381
[尝试次数：1] - [剩余数量：622] - [当前时间：14:39:10]

[ok] - 66392
[尝试次数：1] - [剩余数量：621] - [当前时间：14:39:10]

[ok] - 66386
[尝试次数：1] - [剩余数量：620] - [当前时间：14:39:10]

[ok] - 66387
[尝试次数：1] - [剩余数量：619] - [当前时间：14:39:10]

[ok] - 66388
[尝试次数：1] - [剩余数量：618] - [当前时间：14:39:11]

[ok] - 66398
[尝试次数：1] - [剩余数量：617] - [当前时间：14:39:11]

[ok] - 66393
[尝试次数：1] - [剩余数量：616] - [当前时间：14:39:11]

[ok] - 66396
[尝试次数：1] - [剩余数量：615] - [当前时间：14:39:11]

[ok] - 66399
[尝试次数：1] - [剩余数量：614] - [当前时间：14:39:11]

[ok] - 66384
[尝试次数：1] - [剩余数

[ok] - 66537
[尝试次数：1] - [剩余数量：478] - [当前时间：14:39:21]

[ok] - 66535
[尝试次数：1] - [剩余数量：477] - [当前时间：14:39:21]

[ok] - 66534
[尝试次数：1] - [剩余数量：476] - [当前时间：14:39:21]

[ok] - 66541
[尝试次数：1] - [剩余数量：475] - [当前时间：14:39:21]

[ok] - 66527
[尝试次数：1] - [剩余数量：474] - [当前时间：14:39:21]

[ok] - 66544
[尝试次数：1] - [剩余数量：473] - [当前时间：14:39:21]

[ok] - 66558
[尝试次数：1] - [剩余数量：472] - [当前时间：14:39:21]

[ok] - 66554
[尝试次数：1] - [剩余数量：471] - [当前时间：14:39:21]

[ok] - 66547
[尝试次数：1] - [剩余数量：470] - [当前时间：14:39:22]

[ok] - 66551
[尝试次数：1] - [剩余数量：469] - [当前时间：14:39:22]

[ok] - 66542
[尝试次数：1] - [剩余数量：468] - [当前时间：14:39:22]

[ok] - 66552
[尝试次数：1] - [剩余数量：467] - [当前时间：14:39:22]

[ok] - 66555
[尝试次数：1] - [剩余数量：466] - [当前时间：14:39:22]

[ok] - 66559
[尝试次数：1] - [剩余数量：465] - [当前时间：14:39:22]

[ok] - 66543
[尝试次数：1] - [剩余数量：464] - [当前时间：14:39:22]

[ok] - 66545
[尝试次数：1] - [剩余数量：463] - [当前时间：14:39:22]

[ok] - 66546
[尝试次数：1] - [剩余数量：462] - [当前时间：14:39:22]

[ok] - 66549
[尝试次数：1] - [剩余数量：461] - [当前时间：14:39:22]

[ok] - 66550
[尝试次数：1] - [剩余数

[ok] - 66699
[尝试次数：1] - [剩余数量：323] - [当前时间：14:39:33]

[ok] - 66698
[尝试次数：1] - [剩余数量：322] - [当前时间：14:39:33]

[ok] - 66690
[尝试次数：1] - [剩余数量：321] - [当前时间：14:39:33]

[ok] - 66691
[尝试次数：1] - [剩余数量：320] - [当前时间：14:39:33]

[ok] - 66702
[尝试次数：1] - [剩余数量：319] - [当前时间：14:39:33]

[ok] - 66686
[尝试次数：1] - [剩余数量：318] - [当前时间：14:39:34]

[ok] - 66717
[尝试次数：1] - [剩余数量：317] - [当前时间：14:39:34]

[ok] - 66704
[尝试次数：1] - [剩余数量：316] - [当前时间：14:39:34]

[ok] - 66710
[尝试次数：1] - [剩余数量：315] - [当前时间：14:39:34]

[ok] - 66718
[尝试次数：1] - [剩余数量：314] - [当前时间：14:39:34]

[ok] - 66719
[尝试次数：1] - [剩余数量：313] - [当前时间：14:39:34]

[ok] - 66712
[尝试次数：1] - [剩余数量：312] - [当前时间：14:39:34]

[ok] - 66707
[尝试次数：1] - [剩余数量：311] - [当前时间：14:39:34]

[ok] - 66713
[尝试次数：1] - [剩余数量：310] - [当前时间：14:39:34]

[ok] - 66716
[尝试次数：1] - [剩余数量：309] - [当前时间：14:39:34]

[ok] - 66714
[尝试次数：1] - [剩余数量：308] - [当前时间：14:39:34]

[ok] - 66705
[尝试次数：1] - [剩余数量：307] - [当前时间：14:39:34]

[ok] - 66721
[尝试次数：1] - [剩余数量：306] - [当前时间：14:39:34]

[ok] - 66706
[尝试次数：1] - [剩余数

[ok] - 66864
[尝试次数：1] - [剩余数量：170] - [当前时间：14:39:46]

[ok] - 66855
[尝试次数：1] - [剩余数量：169] - [当前时间：14:39:46]

[ok] - 66863
[尝试次数：1] - [剩余数量：168] - [当前时间：14:39:46]

[ok] - 66850
[尝试次数：1] - [剩余数量：167] - [当前时间：14:39:46]

[ok] - 66852
[尝试次数：1] - [剩余数量：166] - [当前时间：14:39:46]

[ok] - 66856
[尝试次数：1] - [剩余数量：165] - [当前时间：14:39:46]

[ok] - 66859
[尝试次数：1] - [剩余数量：164] - [当前时间：14:39:46]

[ok] - 66857
[尝试次数：1] - [剩余数量：163] - [当前时间：14:39:46]

[ok] - 66865
[尝试次数：1] - [剩余数量：162] - [当前时间：14:39:46]

[ok] - 66854
[尝试次数：1] - [剩余数量：161] - [当前时间：14:39:46]

[ok] - 66860
[尝试次数：1] - [剩余数量：160] - [当前时间：14:39:46]

[ok] - 66866
[尝试次数：1] - [剩余数量：159] - [当前时间：14:39:47]

[ok] - 66867
[尝试次数：1] - [剩余数量：158] - [当前时间：14:39:47]

[ok] - 66874
[尝试次数：1] - [剩余数量：157] - [当前时间：14:39:47]

[ok] - 66878
[尝试次数：1] - [剩余数量：156] - [当前时间：14:39:47]

[ok] - 66868
[尝试次数：1] - [剩余数量：155] - [当前时间：14:39:47]

[ok] - 66871
[尝试次数：1] - [剩余数量：154] - [当前时间：14:39:47]

[ok] - 66873
[尝试次数：1] - [剩余数量：153] - [当前时间：14:39:47]

[ok] - 66875
[尝试次数：1] - [剩余数

[ok] - 83350
[尝试次数：1] - [剩余数量：14] - [当前时间：14:39:59]

[ok] - 83157
[尝试次数：1] - [剩余数量：13] - [当前时间：14:39:59]

[ok] - 83224
[尝试次数：1] - [剩余数量：12] - [当前时间：14:39:59]

[ok] - 83228
[尝试次数：1] - [剩余数量：11] - [当前时间：14:39:59]

[ok] - 83144
[尝试次数：1] - [剩余数量：10] - [当前时间：14:39:59]

[ok] - 83364
[尝试次数：1] - [剩余数量：9] - [当前时间：14:39:59]

[ok] - 83158
[尝试次数：1] - [剩余数量：8] - [当前时间：14:39:59]

[ok] - 83177
[尝试次数：1] - [剩余数量：7] - [当前时间：14:39:59]

[ok] - 83372
[尝试次数：1] - [剩余数量：6] - [当前时间：14:39:59]

[ok] - 83186
[尝试次数：1] - [剩余数量：5] - [当前时间：14:39:59]

[ok] - 83222
[尝试次数：1] - [剩余数量：4] - [当前时间：14:39:59]

[ok] - 83220
[尝试次数：1] - [剩余数量：3] - [当前时间：14:39:59]

[ok] - 83294
[尝试次数：1] - [剩余数量：2] - [当前时间：14:39:59]

[ok] - 83272
[尝试次数：1] - [剩余数量：1] - [当前时间：14:39:59]

[ok] - 83365
[尝试次数：1] - [剩余数量：0] - [当前时间：14:39:59]

[ok] - 83373
[尝试次数：1] - [剩余数量：0] - [当前时间：14:39:59]

[ok] - 84545
[尝试次数：1] - [剩余数量：0] - [当前时间：14:40:00]

[ok] - 84512
[尝试次数：1] - [剩余数量：0] - [当前时间：14:40:00]

[ok] - 86062
[尝试次数：1] - [剩余数量：0] - [当前时间：14:40:00]

[ok] - 